In [9]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import SimpleITK as sitk
import numpy as np
%matplotlib inline

In [10]:
def swc_to_dataframe( file_name ) :
    col_names = ['id', 'type', 'x', 'y', 'z', 'radius', 'parent', 'seg_id', 'level', 'mode', 'timestamp', 'TFresindex']
    df = pd.read_csv( file_name, sep=r"\s+", comment = "#", names = col_names, index_col = 'id')
    return df

In [11]:
def filter_swc( df, type ) :
    dd = df[df['type']==type]
    return dd

In [18]:
#model_directory = "/allen/programs/celltypes/workgroups/mousecelltypes/_UPENN_fMOST"
model_directory = "C:/Users/lydia/OneDrive/Work/DigitalAssets/_UPENN_fMOST"

# Mouse CCF with correct headers
image_directory = os.path.join( model_directory, "..", "converted_mouse_ccf", "average_template")

sample_id = '192343'
swc_directory = os.path.join( model_directory, "HUSTReproduction_9_7_2021", "Sandbox", "HUST_REGISTRATION_AllCells_in_micron_registered", sample_id )
output_directory = os.path.join( model_directory, "HUSTReproduction_9_7_2021","Sandbox", "HUST_REGISTRATION_AllCells_in_micron_registered_thumbanail", sample_id)

In [19]:
#
# component types
#
types = {}
types['soma'] = {'type_id': 1, 'color': 'black'}
types['axon'] = {'type_id': 2, 'color': 'cyan'}
types['basal dendrite'] = {'type_id': 3, 'color': 'orangered'}
types['apical dendrite'] = {'type_id': 4, 'color': 'maroon'}

#
# views for swc
#
views = {}
views['coronal'] = {'order': 0, 'x': 'y', 'y': 'x'} #z
views['sagittal'] = {'order': 1, 'x': 'z', 'y' : 'x'} #y
views['horizontal'] = {'order': 2, 'x': 'z', 'y' : 'y'} #x

#
# image slices
#
slices = {}
slices['coronal'] = {'swc': 'z', 'image' : 'x', 'transpose' : True }
slices['sagittal'] = {'swc': 'y', 'image' : 'z', 'transpose' : False }
slices['horizontal'] = {'swc' : 'x', 'image' : 'y', 'transpose' : False }

#
# scale from image pixel index to SWC coordinate
#
scales = {}
scales['z'] = 10
scales['x'] = 10
scales['y'] = 10

shifts = {}
shifts['z'] = 0
shifts['x'] = -500
shifts['y'] = 0

In [20]:
#
# Create morphology thumbnail for each SWC file in the swc directory
#
#

image_file = os.path.join(image_directory, "average_template_10.nii.gz" )
image = sitk.ReadImage( image_file )
print(image.GetSize())

for swc_file in glob.glob(os.path.join(swc_directory, "*.swc")):
    
    bb = os.path.basename( swc_file )
    
    # find the associated brain file
    bname = bb.split('_')[0]
    
    # set the output thumbnail png file name
    cname = bb.split('.')[0]
    thumnail_file = os.path.join( output_directory, "%s.png" % cname )
    
    print((cname,bname))
    
    #
    # read swc components into dataframe
    #
    df = swc_to_dataframe( swc_file )


    #
    # scale swc coordinates to match image volume
    #
    for sk, sv in shifts.items() :
        df[sk] = df[sk] + sv

    for sk, sv in scales.items() :
        df[sk] = df[sk] / sv
        

    #
    # get background image from soma location
    #

    filtered = filter_swc(df,types['soma']['type_id'])
    
    if len(filtered) == 0 :
        print("no soma found")
        continue
        
    soma = filtered.iloc[0]

    #
    #  plot swc components onto of image at soma location
    #
    plt.rcParams["figure.figsize"] = (30,8)

    fig, axs = plt.subplots(1,3)
    fig.suptitle(os.path.basename(swc_file))

    for vk, vv in views.items() :

        idx = int(round(soma[slices[vk]['swc']],0))

        if slices[vk]['image'] == 'x' :
            bkg = sitk.GetArrayViewFromImage( image )[:,:,idx]
        elif slices[vk]['image'] == 'y' :
            bkg = sitk.GetArrayViewFromImage( image )[:,idx,:]
        elif slices[vk]['image'] == 'z' :
            bkg = sitk.GetArrayViewFromImage( image )[idx,:,:]

        if slices[vk]['transpose'] :
            bkg = bkg.transpose()

        axs[vv['order']].imshow( bkg, cmap='gray')

        for tk,tv in types.items() :

            filtered = filter_swc(df, tv['type_id'])

            x = filtered[vv['x']].values
            y = filtered[vv['y']].values

            axs[vv['order']].scatter( x, y, s=0.001, c=tv['color'])

        axs[vv['order']].set_aspect('equal')

    
    plt.savefig(thumnail_file, dpi = 150)
    plt.close("all")
    

(1320, 800, 1140)
('192343_3226-X6435-Y14389_reg', '192343')
('192343_3481-X7225-Y16596_reg', '192343')
('192343_3985-X6283-Y20312_reg', '192343')
('192343_5011-X4797-Y11341_reg', '192343')
('192343_5289-X4672-Y10733_reg', '192343')
('192343_5384-X4338-Y12696_reg', '192343')
('192343_5525-X6515-Y25071_reg', '192343')
('192343_5742-X3633-Y12817_reg', '192343')
('192343_6093-X5680-Y24926_reg', '192343')
('192343_6135-X5300-Y24595_reg', '192343')
('192343_6211-X5398-Y7060_reg', '192343')
('192343_6777-X4052-Y23666_reg', '192343')
('192343_7174-X3824-Y23571_reg', '192343')
('192343_7524-X5744-Y25593_reg', '192343')
('192343_8154-X4030-Y21534_reg', '192343')
('192343_8460-X8097-Y5617_reg', '192343')
('192343_DoneCellSoma', '192343')
no soma found
('192343_somaMarkers', '192343')
no soma found
